<h1 align="center">
  <a href="https://uptrain.ai">
    <img width="300" src="https://user-images.githubusercontent.com/108270398/214240695-4f958b76-c993-4ddd-8de6-8668f4d0da84.png" alt="uptrain">
  </a>
</h1>

<h1 style="text-align: center;">Automated LLM Finetuning with UpTrain</h1>

In [23]:
# !pip install transformers weightwatcher

In [1]:
import torch
# from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
# from transformers import BertTokenizer, BertForSequenceClassification, BertPreTrainedModel
# from sklearn.model_selection import train_test_split

from helper_funcs import BertForSequenceClassificationWithIntermediateLayer, get_train_val_dataloader

import pandas as pd
# import numpy as np
# from typing import List, Optional, Tuple, Union

# from tqdm import trange
# import random
# import weightwatcher as ww
import uptrain.v0 as uptrain

# import warnings
# warnings.filterwarnings("ignore")

/Users/suyashharlalka/miniconda3/envs/ml/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
PyTorch is available but CUDA is not. Defaulting to NumPy for SVD


#### Download the SPAM collection Dataset

In [25]:
# !wget 'https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip'

In [26]:
# !unzip -o smsspamcollection.zip

In [2]:
file_path = './SMSSpamCollection'
df = pd.DataFrame({'label':int(), 'text':str()}, index = [])
with open(file_path) as f:
    for line in f.readlines():
        split = line.split('\t')
        df = df.append({'label': 1 if split[0] == 'spam' else 0,
                    'text': split[1]},
                    ignore_index = True)
df.head()

,label,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...\n
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


#### Define helper functions

In [28]:
# import torch.nn as nn
# from transformers import BertModel
# from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss
# from transformers.modeling_outputs import SequenceClassifierOutput

In [3]:
model = BertForSequenceClassificationWithIntermediateLayer.from_pretrained('bert-base-uncased', num_labels = 2)
device = ('cuda' if torch.cuda.is_available() else 'cpu')
if device == 'cpu':
    # check for mps
    if torch.backends.mps.is_available():
        device = 'mps'
        
device = torch.device(device)
model.to(device)
train_dataloader, val_dataloader = get_train_val_dataloader(df, batch_size=32)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassificationWithIntermediateLayer: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassificationWithIntermediateLayer from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassificationWithIntermediateLayer from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceC

In [4]:
print("Training data size: " , len(train_dataloader))
print("Validation data size: " , len(val_dataloader))

Training data size:  140
Validation data size:  35


### Define the UpTrain Config 

In [6]:
config = {"checks": 
    [{'type': uptrain.Statistic.FINETUNE,
    'optimizer': torch.optim.AdamW,
    'train_dataloader': train_dataloader,
    'val_dataloader': val_dataloader,
    'device': device,
    # Define the list of new layers that were added
    'layers': [230],
    'is_automated': False,
    'epochs': 2,
    }],
    "logging_args": {"st_logging": True,
                    "log_data": False},
}

In [7]:
framework = uptrain.Framework(cfg_dict=config)

Deleting the folder:  uptrain_smart_data
Deleting the folder:  uptrain_logs



  👋 Welcome to Streamlit!

  If you’d like to receive helpful onboarding emails, news, offers, promotions,
  and the occasional swag, please enter your email address below. Otherwise,
  leave this field blank.

  Email:  

In [ ]:
framework.log(inputs={'model': [model]})

In [34]:
config["checks"][0].update({'is_automated': True})
config["logging_args"].update({'logging_folder': 'automated_logs'})
model = BertForSequenceClassificationWithIntermediateLayer.from_pretrained('bert-base-uncased', num_labels=2)
model.to(device)
framework2 = uptrain.Framework(cfg_dict=config)
framework2.log(inputs={'model': [model]})

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassificationWithIntermediateLayer: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassificationWithIntermediateLayer from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassificationWithIntermediateLayer from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceC

Deleting the folder:  uptrain_smart_data
Deleting the folder:  uptrain_logs


[0]